# 3.1 Cross-validation:evaluating estimator performance
在训练集上测试参数的学习效果，会得到很好的结果，因为都是在同一个数据集上。但是训练好的参数在没有见过的数据集上效果可能会不好，这就是**过拟合(over fitting)**。为了避免这种情况，应该在原始的数据集上留出一部分数据作为测试集。

在*scikit-learn*中，使用*train_test_split*就可很方便地分割出测试集。

下面，用iris数据集来示范。

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

iris=datasets.load_iris()
iris.data.shape,iris.target.shape

((150L, 4L), (150L,))

现在我们可以将数据集中的40%分成测试集。

In [19]:
x_train,x_test,y_train,y_test=train_test_split(
    iris.data,iris.target,test_size=0.4,random_state=0)
#random_state代表随机数种子，不填的时候每次产生的随机数都不一样；填了某个数就代表选择了
#该序号的随机数序列，每次产生的随机数都是一样的（填0每次产生的也一样）
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((90L, 4L), (90L,), (60L, 4L), (60L,))

但是这样仍然会有过拟合的问题（？没看懂）
>there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 

为了解决这个问题，需要在数据集上留出另一部分作为**验证集（validation set)**。训练集上训练结束后，在验证集上进行验证，当结果看起来成功以后，就可以在测试集上进行最终的评估。

但是，将数据集分成三部分后，可以用来学习模型的数目大大减少了，因此结果可能会依赖于特定随机选取的训练集和验证集对。

解决这个问题可以用**交叉验证法（cross-validation,CV)**：依然需要测试集来进行最终的评估，但是不需要验证集了。在基本的方法，**k折交叉验证（k-fold CV)**中，训练集被分为k个小的子集。

大致过程如下：
* 用k-1个子集作为训练集
* 剩下的那个子集作为测试集

k折交叉验证通常需要随机使用不同的划分重复p次，最终评估的结果是折p次k折交叉验证结果的均值，常见的有**10次10折交叉验证**。

这种方法**计算消耗大**，但是**没有浪费很多数据**，在数据集很小的时候这个优点更加突出。

## 3.1.1 Computing cross-validated metrics

使用**cross-validation**的最简单的方法就是调用*cross_val_score*函数。

下面的例子演示了如何在iris数据集上评估线性SVM模型的准确度。

In [21]:
from sklearn.model_selection import cross_val_score
clf=svm.SVC(kernel='linear',C=1)
scores=cross_val_score(clf,iris.data,iris.target,cv=5)#5-fold
scores

array([ 0.96666667,  1.        ,  0.96666667,  0.96666667,  1.        ])

默认的在每个CV中计算分数的方法是*score*，可以通过*scoring*参数来更改。

In [22]:
from sklearn import metrics
scores=cross_val_score(
    clf,iris.data,iris.target,cv=5,scoring='f1_macro')#以后详细阐述scoring参数含义
scores

array([ 0.96658312,  1.        ,  0.96658312,  0.96658312,  1.        ])

当cv是整数的时候，*cross_val_score*默认使用**KFold**或者**StratifiedKFold**，后者在estimator是**ClassifierMixin**中派生而来时默认使用。

同样，可以通过传输交叉验证迭代器来使用其他的交叉验证方法，例如：

In [24]:
from sklearn.model_selection import ShuffleSplit
n_samples=iris.data.shape[0]
cv=ShuffleSplit(n_splits=3,test_size=0.3,random_state=0)
cross_val_score(clf,iris.data,iris.target,cv=cv)

array([ 0.97777778,  0.97777778,  1.        ])

正如在留出的数据上测试训练好的预测器很重要，**预处理**（如归一化，特征选择等等）以及**数据变形（data transformations)**都应该在训练集中学习好然后应用到留出的数据上来进行预测。

通过在cv中应用**Pipeline**让生成estimator更简单了。

### 3.1.1.1 Obtaining predictions by cross-validation

*cross_val_predict*函数和*cross_val_score*具有相似的接口，但是前者只会返回训练集中的预测结果。

预测结果可以用来评估分类器：

In [25]:
from sklearn.model_selection import cross_val_predict
predicted=cross_val_predict(clf,iris.data,iris.target,cv=10)
metrics.accuracy_score(iris.target,predicted)

0.97333333333333338

## 3.1.2 Cross validation iterators

接下来列举了根据不同的cv方法，utilities产生序号来产生分割的数据集。

## 3.1.3 Cross-validation iterators for i.i.d. data

假设数据是**独立同分布（Independent Identically Distributed，i.i.d.）**的，则所有的样本都来自于同样的产生过程且产生过程对之前产生的样本没有记忆。

接下来的交叉验证器可以用于这种情况。

**注意**
尽管i.i.d.数据是机器学习理论中常用的假设，在实际中很少使用。如果样本是根据时间相关的进程产生的，用时间序列的cv方案<time_series_cv>更安全。

### 3.1.3.1 K-fold

**KFold**将所有的样本分成了相同大小的k组，称作*折（fold）*。

下面是一个四个样本的2折cv例子。

In [27]:
import numpy as np
from sklearn.model_selection import KFold#返回序号

X=['a','b','c','d']
kf=KFold(n_splits=2)#分割的折数
for train,test in kf.split(X):
    print('%s %s'%(train,test))

[2 3] [0 1]
[0 1] [2 3]


每一折都是由两个array组成的：第一个跟*训练集*有关，第二个跟*测试集*有关。这样，可以通过*numpy indexing*来产生*训练/测试集*。

In [29]:
X=np.array([[0.,0.],[1.,1.],[-1.,-1.],[2.,2.]])
y=np.array([0,1,0,1])
X_train,X_test,y_train,y_test=X[train],X[test],y[train],y[test]
X_train,X_test,y_train,y_test

(array([[ 0.,  0.],
        [ 1.,  1.]]), array([[-1., -1.],
        [ 2.,  2.]]), array([0, 1]), array([0, 1]))

### 3.1.3.2 Leave One Out(LOO)

**留一法（LOO)**是一个简单的cv。设数据集包含m个样本，每个训练集都是从所有的样本中取m-1个构成的，剩下的一个样本作为测试集。因此，对m个样本，我们由m个不同的训练集和m个不同的测试集。这种方法没有浪费很多数据，因为只有一个样本从训练集中移除了。

In [30]:
from sklearn.model_selection import LeaveOneOut#返回序号
X=[1,2,3,4]
loo=LeaveOneOut()
for train,test in loo.split(X):
    print ('%s %s' %(train,test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


| Method | build models | trained on samples |
|--------|:------------:|:------------------:|
|K-fold |k        |(k-1)m/k|
|LOO|m|m-1|

假设两种方法中k都不是特别大，k< m ,LOO 比K折cv的计算开销更大；
准确度方面，LOO经常导致高方差；
如果训练集的学习曲线陡峭的话，5或者10折cv会高估泛化误差。

综上，推荐使用**5或者10折cv**而不是LOO。

### 3.1.3.3 Leave P Out(LPO)

**LeavePOut**和**LeaveOneOut**很相似，它从所有的数据集中移除p个样本构成训练集，移除的p个样本构成测试集。对于n个样本来说，这个方法产生了（n,p)训练-测试数据对。不像LeaveOneOut和KFold,测试集在p>1的情况下会**重叠(overlap)**。

下面是一个四个样本的Leave-2-Out的例子：

In [31]:
from sklearn.model_selection import LeavePOut#返回序号
X=np.ones(4)
lpo=LeavePOut(p=2)
for train,test in lpo.split(X):
    print('%s %s'%(train,test))

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


### 3.1.3.4 Random permutations cross-validation a.k.a. Shuffle & Split

**ShuffleSplit**迭代器会产生用户规定数目的独立的训练和测试集分割。样本首先被打乱然后被分成训练和测试集。

In [32]:
from sklearn.model_selection import ShuffleSplit #返回的是分割数据集的序号
X=np.arange(5)
ss=ShuffleSplit(n_splits=3,test_size=.25,random_state=0)#n_splits 分割的次数
for train_index,test_index in ss.split(X):
    print('%s %s'%(train_index,test_index))

[1 3 4] [2 0]
[1 4 3] [0 2]
[4 0 2] [1 3]


## 3.1.4 Cross-validation iterators with stratification based on class labels

当目标的分类分布特别不均衡时，分类会产生问题：例如反例样本是正例样本数目的几倍。在这种情况下，推荐使用分层采样**StratifiedKFold**和**StratifiedSuffleSplit**来保证样本类别的比例相似。

### 3.1.4.1 Stratified k-fold

**StratifiedKFold**是*k-fold*的一个拓展，返回分层的折：每个子集包括差不多比例的目标类别。

下面是一个2类轻微不平衡的10个样本的3折分层cv例子：

In [33]:
from sklearn.model_selection import StratifiedKFold#返回序号
X=np.ones(10)
y=[0,0,0,0,1,1,1,1,1,1]
skf=StratifiedKFold(n_splits=3)
for train,test in skf.split(X,y):
    print('%s %s'%(train,test))

[2 3 6 7 8 9] [0 1 4 5]
[0 1 3 4 5 8 9] [2 6 7]
[0 1 2 4 5 6 7] [3 8 9]


### 3.1.4.2 Stratified Shuffle Split

**StratifiedShuffleSplit**是*ShuffleSplit*的一个拓展，返回分层的分割集。

In [35]:
from sklearn.model_selection import StratifiedShuffleSplit
X=np.ones(10)
y=[0,0,0,0,1,1,1,1,1,1]
sss=StratifiedShuffleSplit(n_splits=3,test_size=.25,random_state=0)
for train,test in sss.split(X,y):
    print('%s %s'%(train,test))

[3 2 8 4 6 1 5] [9 7 0]
[1 7 8 9 6 0 3] [4 2 5]
[9 0 5 6 2 1 8] [7 4 3]


## 3.1.5 Cross-validation iterators for grouped data

若潜在的产生过程产生了相关的样本组，则i.i.d.假设不成立。

这种数据组是区域特定的。若从很多患者处采集了药物数据，每个患者处都采集了很多样本。这种数据对每个人是相关的。在这个例子中，每个样本的患者序号就是组别。

这种情况下我们想知道特定组中训练好的模型是否能在未见过的组别中泛化很好。我们需要保证验证折中的所有样本的来自组别都没有在对应的训练折中出现。

接下来的cv分裂器可以实现这个。样本的组别通过*groups*这个变量来指定。

### 3.1.5.1 Group k-fold

**GroupKFold**是*k-fold*的一个拓展，保证了同一个组没有在成对的测试集和训练集中出现。例如：数据从不同的目标中采集，每个目标都采集了很多样本；如果模型足够灵活从高度个人特定的特征中学习，在新的目标中泛化就可能失败。**GroupKFold**能检测到这种过拟合的情况。

假设你有3个目标，每个目标存在相关联的数字1-3：

In [40]:
from sklearn.model_selection import GroupKFold#返回序号

X=[0.1,0.2,2.2,2.4,2.3,4.55,5.8,8.8,9,10]
y=['a','b','b','b','c','c','c','d','d','d']
groups=[1,1,1,2,2,2,3,3,3,3]
gkf=GroupKFold(n_splits=2) #n_splits 不能大于组别数目
for train,test in gkf.split(X,y,groups=groups):
    print('%s %s'%(train,test))

[0 1 2 3 4 5] [6 7 8 9]
[6 7 8 9] [0 1 2 3 4 5]


每个目标都在不同的测试折中，相同的目标不会在测试集和训练集中出现。折没有相同的大小，因为数据的均衡。

### Leave One Group Out

**LeaveOneGroupOut**使一种cv方案，根据第三方指定的整数数组留出样本。

每个训练集都由除了一个与特定组相关的样本外的样本组成。

如：在多次实验的情况下，**LeaveOneGroupOut**可以基于不同的实验来产生cv：用除了一个实验的样本外的其他样本来产生训练集：

In [38]:
from sklearn.model_selection import LeaveOneGroupOut#返回序号
X=[1,5,10,50,60,70,80]
y=[0,1,1,2,2,2,2]
groups=[1,1,2,2,3,3,3]
logo=LeaveOneGroupOut()
for train,test in logo.split(X,y,groups=groups):
    print ('%s %s'%(train,test))

[2 3 4 5 6] [0 1]
[0 1 4 5 6] [2 3]
[0 1 2 3] [4 5 6]


** Group k-fold 与 Leave One Group Out 区别**
* Group k-fold 指定k（组）折，可能留出几个组
* Leave One Group Out中只可能留出一个组

### 3.1.5.3 Leave P Groups Out

**LeavePGroupsOut**和**LeaveOneGroupOut**相似，但是移除了与*P*个组相关的样本。

下面是一个*Leave-2-Group Out*的例子：

In [41]:
from sklearn.model_selection import LeavePGroupsOut#返回序号

X=np.arange(6)
y=[1,1,1,2,2,2]
groups=[1,1,2,2,3,3]
lpgo=LeavePGroupsOut(n_groups=2)
for train,test in lpgo.split(X,y,groups=groups):
    print('%s %s'%(train,test))

[4 5] [0 1 2 3]
[2 3] [0 1 4 5]
[0 1] [2 3 4 5]


### 3.1.5.4 Group Shuffle Split

**GroupShuffleSplit**是**ShuffleSplit**和**LeavePGroupsOut**的结合。

In [43]:
from sklearn.model_selection import GroupShuffleSplit#返回序号

X=[0.1,0.2,2.2,2.4,2.3,4.55,5.8,0.001]
y=['a','b','b','b','c','c','c','a']
groups=[1,1,2,2,3,3,4,4]
gss=GroupShuffleSplit(n_splits=4,test_size=0.25,random_state=0)
for train,test in gss.split(X,y,groups=groups):
    print('%s %s'%(train,test))

[0 1 2 3 6 7] [4 5]
[2 3 4 5 6 7] [0 1]
[0 1 2 3 4 5] [6 7]
[0 1 4 5 6 7] [2 3]


这种方法在需要LeavePGroupsOut，但组别的数量太大了导致产生所有可能的P个组别的代价太大了时很有用。

## 3.1.6 Predefined Fold-Splits/Validation-Sets

在一些数据集中，预先定义好的分割训练集和验证集的方法已经存在了。使用**PredifinedSplit**可以使用这些折如寻找超参数时。

例如，使用验证集时，将验证集中所有样本的*test_fold*设为0，其他的样本中设为1。

## 3.1.7 Cross validation of time series data

时间序列的特征是由在时间上相近的观察之间的相关性所描述的。但是，传统的cv方法如*kfold*和*shufflesplit*假设样本是**独立同分布**的，会导致时间序列数据中训练和测试样本的不知原因的相关性。因此，在与训练模型的时间序列最不相似的*将来*观察上评估模型很重要。为了解决这个问题，可以使用**TimeSeriesSplit**。

### 3.1.7.1 Time Series Split

**TimeSeriesSplit**是*k-fold*的一个拓展，第一次返回前k折作为训练集，第k+1折作为测试集。接下来的训练把之前测试集中所有数据加到第一个训练部分中，通常用于训练模型。

这种方法可以用来交叉验证特定时间间隔观察到的时间序列样本。

下面是一个6个样本的3折时间序列cv例子：

In [44]:
from sklearn.model_selection import TimeSeriesSplit#返回序号

X=np.array([[1,2],[3,4],[1,2],[3,4],[1,2],[3,4]])
y=np.array([1,2,3,4,5,6])
tscv=TimeSeriesSplit(n_splits=3)
for train,test in tscv.split(X):
    print('%s %s'%(train,test))

[0 1 2] [3]
[0 1 2 3] [4]
[0 1 2 3 4] [5]
